# Pair Programming ETL III

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [4]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
import ast

In [ ]:
dicc = {"usa": [39.7837304, -100.445882], "australia":[-24.7761086, 134.755], "south africa":[-28.8166236, 24.991639],"new zealand": [-41.5000831, 172.8344077],"papua new guinea": [-5.6816069, 144.2489081] }

In [17]:
class extraer_limpiar:
    
    def __init__(self, lat, lon, pais):
        
        self.lat = lat
        self.lon = lon
        self.pais = pais         

    def extraer_datos(self):
        
        df = pd.DataFrame()
    
        url ="http://www.7timer.info/bin/api.pl?lon={self.lon}&lat={self.lat}&product=meteo&output=json"
    
        response = requests.get(url = url) 
        response.status_code
    
        response_df = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
        response_df["country"] = self.pais
    
        df = pd.concat([df, response_df], axis = 0)
        return df 
    
    def limpiar_rh(self,df):
        
        #df['rh_profile']= df['rh_profile'].apply(ast.literal_eval)

        x = df['rh_profile'].apply(pd.Series) 

        for i in range(len(x.columns)): 
            nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
            valores = list(x[i].apply(pd.Series)["rh"] )
     
            df.insert(i, nombre, valores)
        return df
    
    def limpiar_wind(self,df2):
    
        #df2['wind_profile']= df2['wind_profile'].apply(ast.literal_eval)

        y = df2['wind_profile'].apply(pd.Series)

        for i in range(len(y.columns)): 
            nombre = "winddirection_" + str(y[i].apply(pd.Series)["layer"][0])
            valores = list(y[i].apply(pd.Series)["direction"])

            df2.insert(i, nombre, valores)
    
        return df2


In [18]:
api = extraer_limpiar(39.7837304, -100.445882,"usa")
api

In [22]:
df_extraido = api.extraer_datos()
df_extraido.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 190, 'speed':...",26,-1,12,-9999,none,0,0,190,3,usa
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 210, 'speed':...",26,-4,12,-9999,none,0,0,200,3,usa


In [20]:
df_prueba = api.limpiar_rh(df_extraido)


In [21]:
df_prueba

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,...,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,13,11,11,10,11,13,13,15,15,8,...,26,-1,12,-9999,none,0,0,190,3,usa
1,11,10,9,9,10,11,14,13,10,8,...,26,-4,12,-9999,none,0,0,200,3,usa
2,14,11,10,10,11,12,12,11,8,5,...,26,-1,11,-9999,none,0,0,185,3,usa
3,10,11,11,11,10,10,10,12,9,10,...,26,-1,11,-9999,rain,1,0,190,3,usa
4,11,11,11,11,11,11,11,11,11,11,...,26,-4,11,-9999,rain,2,0,200,2,usa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,12,12,13,7,6,7,6,8,9,-2,...,27,-1,11,-9999,rain,3,0,170,3,usa
60,12,12,12,8,6,6,7,8,9,1,...,27,-1,10,-9999,rain,3,0,165,2,usa
61,11,11,13,7,6,8,8,8,10,4,...,27,-1,10,-9999,rain,3,0,170,2,usa
62,13,12,12,7,5,8,11,12,13,5,...,27,-1,10,-9999,none,3,0,165,3,usa
